<a href="https://colab.research.google.com/github/paulquimbo/nhanes_inferential_2021_23/blob/main/nhanes_inferential.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install pyreadstat
!pip install pandas


In [15]:
import pyreadstat
import pandas
import os

In [18]:
#  checking for xpt file that are non readable with pyreadtat because of formatting issue
folder = "/content/raw/"
for file in os.listdir(folder):
    if file.endswith(".xpt"):
        path = os.path.join(folder, file)
        try:
            df, meta = pyreadstat.read_xport(path)
            print(f"✅ Loaded: {file}")
        except Exception as e:
            print(f"❌ Failed: {file} — {e}")

✅ Loaded: VID_L.xpt
✅ Loaded: HEPB_S_L.xpt
✅ Loaded: PAQ_L.xpt
❌ Failed: DEMO_L.xpt — 'utf-8' codec can't decode byte 0x92 in position 13: invalid start byte
✅ Loaded: KIQ_U_L.xpt
✅ Loaded: BPXO_L.xpt
✅ Loaded: WHQ_L.xpt


In [19]:
df, meta = pyreadstat.read_xport(demo)
print(df.head())

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x92 in position 13: invalid start byte

In [2]:
# Install haven (only needed once)
install.packages("haven")

# Load the library
library(haven)

# Set folder path
folder <- "/content/raw"

# List all .xpt files
xpt_files <- list.files(folder, pattern = "\\.xpt$", full.names = TRUE)

# Loop through and convert each to .csv
for (xpt_file in xpt_files) {
  # Read .xpt file
  data <- read_xpt(xpt_file)

  # Create .csv filename
  csv_file <- sub("\\.xpt$", ".csv", xpt_file)

  # Write to CSV
  write.csv(data, csv_file, row.names = FALSE)

  cat("✅ Converted:", xpt_file, "→", csv_file, "\n")
}

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



✅ Converted: /content/raw/BPXO_L.xpt → /content/raw/BPXO_L.csv 
✅ Converted: /content/raw/DEMO_L.xpt → /content/raw/DEMO_L.csv 
✅ Converted: /content/raw/HEPB_S_L.xpt → /content/raw/HEPB_S_L.csv 
✅ Converted: /content/raw/KIQ_U_L.xpt → /content/raw/KIQ_U_L.csv 
✅ Converted: /content/raw/PAQ_L.xpt → /content/raw/PAQ_L.csv 
✅ Converted: /content/raw/VID_L.xpt → /content/raw/VID_L.csv 
✅ Converted: /content/raw/WHQ_L.xpt → /content/raw/WHQ_L.csv 


In [17]:
# Mapping urls
demo = "/content/raw/DEMO_L.csv"
bp = "/content/raw/BPXO_L.csv"
vitd = "/content/raw/VID_L.csv"
hepb = "/content/raw/HEPB_S_L.csv"
kidney = "/content/raw/KIQ_U_L.csv"
minbehavior = "/content/raw/PAQ_L.csv"
weight = "/content/raw/WHQ_L.csv"

In [1]:
# Using R to convent xpt to csv
library(haven)
data <- read_xpt("/content/raw/DEMO_L.xpt")
write.csv(data, "/content/raw/DEMO_L.csv", row.names = FALSE)